In [72]:
import json
import pandas as pd
from pathlib import Path


In [73]:
# Defining paths
GROUP_DATA_DIR = Path("/Users/harshul/projects/kgwebsite/group-data")

MEMBERS_DIR_PATH = GROUP_DATA_DIR / 'members/'
WEBSITE_DATA_PATH = GROUP_DATA_DIR / 'website_data/'
CONTENT_DIR_PATH = WEBSITE_DATA_PATH / 'content'


In [74]:
# creating dataframe for education and experiences
def parse_education_experience_df(member_dir):
    member_json_dir = member_dir / 'jsons'
    education_experience_list = []

    if (member_experiences_dir := (member_json_dir / 'experiences.json')).exists():
        with open(member_experiences_dir, 'r') as f:
            education_experience_list += json.load(f)
    else:
        print(member_experiences_dir)
    
    if (member_education_dir := (member_json_dir / 'education.json')).exists():
        with open(member_education_dir, 'r') as f:
            education_experience_list += json.load(f)
    
    return pd.DataFrame(education_experience_list)
# To see if member is current member and their roles
member_records = []

def extract_member_academic_role(education_experience_df):
    current_academic_role = ""  
    is_current_member = False
    if 'end_date' not in education_experience_df.columns:
        education_experience_df['end_date'] = None
    filtered_e_e_df = education_experience_df[education_experience_df.end_date.isna()]
    if 'institution' in filtered_e_e_df.columns:
        filtered_e_e_df = filtered_e_e_df[filtered_e_e_df['institution'] == 'Michigan State University']
    else:

        print("No 'institution' column found in dataframe.")
        filtered_e_e_df = pd.DataFrame()

    
    

    if len(filtered_e_e_df) == 0: 
       pass
        
    
    elif len(filtered_e_e_df) == 1:
        
        role = filtered_e_e_df.iloc[0].get('role', None)  
        degree = filtered_e_e_df.iloc[0].get('degree', None)  
        if role in ['Assistant Professor', 'Professor']:
            current_academic_role = 'Professor'

        elif role in ['Visiting Researcher', 'Postdoctoral Researcher']:
            current_academic_role = 'Postdoctoral Researcher'  
  
        elif degree in ['Masters', 'PhD']:
            current_academic_role = 'Graduate Student' 
  
        elif degree == 'Bachelors':

            current_academic_role = 'Graduate Student' if filtered_e_e_df.iloc[0].get('degree') is None else 'Undergraduate Student'

        is_current_member = True

    else:
        raise ValueError('Not sure what happened multiple positions at MSU')
        
    return current_academic_role, is_current_member

# for creating two lists

current_people_page_list = []
alumni_people_page_list = []


for member_dir in MEMBERS_DIR_PATH.glob('*'):
    if not (member_info_fname := member_dir / 'info.json').exists():
        continue
    else:
        member_info = json.load(open(member_info_fname, 'r'))
    education_experience_df = parse_education_experience_df(member_dir)
    current_academic_role, is_current_member = extract_member_academic_role(education_experience_df)

    
    first_name = member_info['first_name']
    last_name = member_info['last_name']
    nickname = member_info.get('nick_name', None)
    id = member_info['id']
    image_path = member_info['image_path']  
    cover_image_path = member_info['cover_image_path']   

    name = f'{nickname if nickname else first_name} {last_name}'

    if is_current_member:
        current_people_page_list.append({"Name": name, "academic_role": current_academic_role, "ID": id})
    else:
        alumni_people_page_list.append({"Name": name, "academic_role": current_academic_role, "ID": id})




/Users/harshul/projects/kgwebsite/group-data/members/bea_lu/jsons/experiences.json
No 'institution' column found in dataframe.


In [75]:
current_people_page_list

[{'Name': 'Josh Shields',
  'academic_role': 'Graduate Student',
  'ID': 'media/images/cover.jpg'},
 {'Name': 'Anirban  Dutta',
  'academic_role': 'Postdoctoral Researcher',
  'ID': 'media/images/cover.jpg'},
 {'Name': 'Erin Visser',
  'academic_role': 'Undergraduate Student',
  'ID': 'media/images/cover.jpg'},
 {'Name': 'Vicente  Amado Olivo',
  'academic_role': 'Graduate Student',
  'ID': 'media/images/cover.jpg'},
 {'Name': 'Yuki Matsumura',
  'academic_role': 'Graduate Student',
  'ID': 'media/images/cover.jpg'},
 {'Name': 'Andrew Fullard',
  'academic_role': '',
  'ID': 'media/images/cover.jpg'},
 {'Name': 'Hayden Monk',
  'academic_role': 'Undergraduate Student',
  'ID': 'media/images/cover.jpg'},
 {'Name': 'Bea Lu',
  'academic_role': 'Undergraduate Student',
  'ID': 'media/images/cover.jpg'},
 {'Name': "Jack O'Brien",
  'academic_role': 'Graduate Student',
  'ID': 'media/images/cover.jpg'},
 {'Name': 'Cecelia Powers',
  'academic_role': 'Undergraduate Student',
  'ID': 'media/i

In [76]:
alumni_people_page_list

[{'Name': 'Isaac Smith', 'academic_role': '', 'ID': 'media/images/cover.jpg'},
 {'Name': 'Atharva Arya', 'academic_role': '', 'ID': 'media/images/cover.jpg'},
 {'Name': 'Tripp Dow', 'academic_role': '', 'ID': 'media/images/cover.jpg'},
 {'Name': 'Sona Chitchyan',
  'academic_role': '',
  'ID': 'media/images/cover.jpg'},
 {'Name': 'Iliomar Rodriguez-Ramos',
  'academic_role': '',
  'ID': 'media/images/cover.jpg'},
 {'Name': 'Jaladh Singhal',
  'academic_role': '',
  'ID': 'media/images/cover.jpg'},
 {'Name': 'Template Person',
  'academic_role': '',
  'ID': 'media/images/cover.jpg'},
 {'Name': 'Kevin Cawley', 'academic_role': '', 'ID': 'media/images/cover.jpg'},
 {'Name': 'Alexander Grunewald',
  'academic_role': '',
  'ID': 'media/images/cover.jpg'},
 {'Name': 'Harshul Gupta',
  'academic_role': '',
  'ID': 'media/images/cover.jpg'}]

In [77]:
member_dir

PosixPath('/Users/harshul/projects/kgwebsite/group-data/members/wolfgang_kerzendorf')

In [78]:
filtered_e_e_df = education_experience_df[education_experience_df.end_date.isna()]

In [79]:
filtered_e_e_df.iloc[0]

institution    Michigan State University
group                         kerzendorf
city                        East Lansing
state                           Michigan
country                               US
role                 Assistant Professor
start_date                    2019-08-15
end_date                             NaN
degree                               NaN
subject                              NaN
Name: 0, dtype: object

In [80]:
parse_education_experience_df(member_dir)

,institution,group,city,state,country,role,start_date,end_date,degree,subject
0,Michigan State University,kerzendorf,East Lansing,Michigan,US,Assistant Professor,2019-08-15,NaN,NaN,NaN
1,New York University,NaN,New York,New York,US,Senior Research Associate,2019-01-01,2019-12-31,NaN,NaN
2,European Southern Observatory,NaN,Garching,Bavaria,DE,ESO Fellow,2014-10-01,2018-12-31,NaN,NaN
3,University of Toronto,NaN,Toronto,Ontario,Canada,NaN,2011-10-01,2014-09-30,Postdoctoral Fellow,NaN
4,Australian National University,NaN,Canberra,,Australia,NaN,2007-02-01,2011-12-18,PhD,Physics and Astronomy
